# Forecasting Pipeline

# Global LGBM Forecaster

In [21]:
import numpy as np
import pandas as pd
from datetime import timedelta
from sqlalchemy import text
import hashlib


In [22]:


# Try LightGBM; fallback to XGBoost if not available
try:
    import lightgbm as lgb
    _USE_LGBM = True
except Exception:
    from xgboost import XGBRegressor
    _USE_LGBM = False

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# Your DB util
from database.db_utils import get_db_connection

# Your existing feature helpers
from forecasting.data_preprocessing import preprocess_data, create_lagged_features

# Controls
HIST_DAYS_PANEL   = 150   # how many days of history per asof to build training rows
MIN_ROWS_PANEL    = 400   # minimum total rows to fit global model
GROUP_COL         = "pool_group"  # <-- set this to your dynamic group column name


In [23]:
def _count_actual_history(panel_df: pd.DataFrame, pid: str, asof_norm: pd.Timestamp) -> int:
    return int(
        panel_df.loc[
            (panel_df['pool_id'] == pid) &
            (panel_df['date'] <= asof_norm) &
            (panel_df['actual_apy'].notna())
        ].shape[0]
    )

def _baseline_from_actual(panel_df: pd.DataFrame, pid: str, asof_norm: pd.Timestamp) -> float | None:
    hist = (panel_df.loc[
        (panel_df['pool_id'] == pid) &
        (panel_df['date'] <= asof_norm) &
        (panel_df['actual_apy'].notna()),
        'actual_apy'
    ].sort_index())
    if len(hist) >= 2:
        return float(hist.tail(2).mean())
    return None

In [24]:
def fetch_panel_history(asof: pd.Timestamp, pool_ids: list, days:int=HIST_DAYS_PANEL,
                        group_col: str = GROUP_COL) -> pd.DataFrame:
    """
    Read-only: fetch last `days` of history up to `asof` for given pools.
    Returns tidy df with columns:
      date, pool_id, apy_7d, actual_apy, tvl_usd, eth_open, btc_open, gas_price_gwei, group_col
    """
    t = pd.Timestamp(asof)
    t = t.tz_localize('UTC') if t.tz is None else t.tz_convert('UTC')
    start = (t - pd.Timedelta(days=days)).normalize()

    engine = get_db_connection()
    q = f"""
        SELECT
            date,
            pool_id,
            rolling_apy_7d AS apy_7d,
            actual_apy,
            actual_tvl     AS tvl_usd,
            eth_open,
            btc_open,
            gas_price_gwei,
            {group_col}    AS {group_col}
        FROM pool_daily_metrics
        WHERE pool_id = ANY(:pool_ids)
          AND date >= :start_date
          AND date <= :asof_date
        ORDER BY date ASC
    """
    with engine.connect() as conn:
        df = pd.read_sql(
            text(q), conn,
            params={
                "pool_ids": pool_ids,
                "start_date": start.tz_convert('UTC').to_pydatetime(),
                "asof_date":  t.tz_convert('UTC').to_pydatetime()
            }
        )
    if df.empty:
        return df

    df['date'] = pd.to_datetime(df['date'], utc=True).dt.normalize()
    return df



In [25]:
def add_neighbor_features(panel_df: pd.DataFrame, group_col: str = None) -> pd.DataFrame:
    """
    For each (date, pool), add group-level neighbor stats.

    Intended usage: predicting t+1 actual_apy using features at t.
    We provide both same-day (t) and past-only (t-1) variants to let you choose.

    Features (suffix _nbr):
      - group_tvl_sum_t_nbr                 (sum TVL at t)
      - group_apy_mean_t_nbr / median / std (based on apy_7d at t)
      - tvl_share_nbr                       (pool TVL share within its group at t)
      - apy_rank_nbr                        (normalized rank of apy_7d within its group at t, 0..1)
      - grp_ex_mean_t_nbr                   (group mean apy_7d at t excluding the pool)
      - grp_ex_mean_7d_nbr                  (7d rolling mean of grp_ex_mean_t by group)
      - *_lag1 counterparts for past-only neighbor stats (computed from t-1)

    Notes:
      - If `group_col` is None or missing, a single group 'ALL' is assumed (no leakage risk).
      - This function uses only columns: ['date','pool_id','apy_7d','tvl_usd', group_col].
        Make sure they exist upstream (we’ll create empties if missing to avoid crashes).
    """
    df = panel_df.copy()

    # Ensure required columns exist
    req = ['date', 'pool_id', 'apy_7d', 'tvl_usd']
    for c in req:
        if c not in df.columns:
            df[c] = np.nan

    # Handle grouping column
    if group_col is None:
        _grp_col = '__ALL__'
        df[_grp_col] = 'ALL'
    else:
        _grp_col = group_col
        if _grp_col not in df.columns:
            # If missing, create a single group to stay robust.
            df[_grp_col] = 'ALL'

    # Normalize dates (daily) and sort
    df['date'] = pd.to_datetime(df['date'], utc=True).dt.normalize()
    df = df.sort_values(['date', _grp_col, 'pool_id'])

    # -------- SAME-DAY (t) group stats (OK for t+1 forecasts) --------
    grp_t = df.groupby(['date', _grp_col], dropna=False)

    group_tvl_sum_t   = grp_t['tvl_usd'].transform('sum')    .rename('group_tvl_sum_t_nbr')
    group_apy_mean_t  = grp_t['apy_7d'].transform('mean')    .rename('group_apy_mean_t_nbr')
    group_apy_median_t= grp_t['apy_7d'].transform('median')  .rename('group_apy_median_t_nbr')
    group_apy_std_t   = grp_t['apy_7d'].transform('std')     .rename('group_apy_std_t_nbr')

    df = pd.concat([df, group_tvl_sum_t, group_apy_mean_t, group_apy_median_t, group_apy_std_t], axis=1)

    # TVL share inside group at t
    denom = df['group_tvl_sum_t_nbr'].replace(0, np.nan)
    df['tvl_share_nbr'] = (df['tvl_usd'] / denom).fillna(0.0)

    # Normalized rank (0..1) of apy_7d within group at t (includes pool itself)
    def _rank_norm(x):
        r = x.rank(method='average', na_option='keep')
        return (r - 1) / max(len(r) - 1, 1)
    df['apy_rank_nbr'] = grp_t['apy_7d'].transform(_rank_norm)

    # Exclude-this-pool group mean at t
    group_count_t = grp_t['apy_7d'].transform('count').rename('group_count_t_nbr')
    group_sum_t   = grp_t['apy_7d'].transform('sum')  .rename('group_sum_t_nbr')
    df = pd.concat([df, group_count_t, group_sum_t], axis=1)

    excl_mean = (df['group_sum_t_nbr'] - df['apy_7d']) / (df['group_count_t_nbr'] - 1).replace(0, np.nan)
    df['grp_ex_mean_t_nbr'] = excl_mean.fillna(df['group_apy_mean_t_nbr'])

    # Build group-daily series for rolling calcs (dedup per (date, group))
    g_daily = (
        df[['date', _grp_col, 'grp_ex_mean_t_nbr']]
        .drop_duplicates(['date', _grp_col])
        .sort_values(['date', _grp_col])
    )

    # 7d rolling of excl-mean (per group)
    g_daily['grp_ex_mean_7d_nbr'] = (
        g_daily.groupby(_grp_col)['grp_ex_mean_t_nbr']
               .transform(lambda s: s.rolling(7, min_periods=3).mean())
    )

    df = df.merge(g_daily[['date', _grp_col, 'grp_ex_mean_7d_nbr']],
                  on=['date', _grp_col], how='left')

    # -------- PAST-ONLY (t-1) variants (for ultra-conservative setups) --------
    # Collapse same-day stats to (date, group), then shift by 1 day per group.
    g_same = (
        df[['date', _grp_col,
            'group_tvl_sum_t_nbr', 'group_apy_mean_t_nbr', 'group_apy_median_t_nbr',
            'group_apy_std_t_nbr', 'grp_ex_mean_t_nbr', 'grp_ex_mean_7d_nbr']]
        .drop_duplicates(['date', _grp_col])
        .sort_values(['date', _grp_col])
        .copy()
    )

    for col in ['group_tvl_sum_t_nbr', 'group_apy_mean_t_nbr', 'group_apy_median_t_nbr',
                'group_apy_std_t_nbr', 'grp_ex_mean_t_nbr', 'grp_ex_mean_7d_nbr']:
        g_same[col + '_lag1'] = (
            g_same.groupby(_grp_col)[col].shift(1)
        )

    df = df.merge(
        g_same[['date', _grp_col] + [c + '_lag1' for c in
               ['group_tvl_sum_t_nbr','group_apy_mean_t_nbr','group_apy_median_t_nbr',
                'group_apy_std_t_nbr','grp_ex_mean_t_nbr','grp_ex_mean_7d_nbr']]],
        on=['date', _grp_col],
        how='left'
    )

    # Final NA handling for neighbor block
    fill_cols = [
        'group_tvl_sum_t_nbr','group_apy_mean_t_nbr','group_apy_median_t_nbr','group_apy_std_t_nbr',
        'tvl_share_nbr','apy_rank_nbr','grp_ex_mean_t_nbr','grp_ex_mean_7d_nbr',
        'group_tvl_sum_t_nbr_lag1','group_apy_mean_t_nbr_lag1','group_apy_median_t_nbr_lag1',
        'group_apy_std_t_nbr_lag1','grp_ex_mean_t_nbr_lag1','grp_ex_mean_7d_nbr_lag1'
    ]
    for c in fill_cols:
        if c in df.columns:
            df[c] = df[c].fillna(0.0)

    return df

In [26]:
EXOG_BASE = ['eth_open', 'btc_open', 'gas_price_gwei', 'tvl_usd', 'apy_7d']

LAG_SETS  = {
    'eth_open':        [7, 30],
    'btc_open':        [7, 30],
    'gas_price_gwei':  [7, 30],
    'tvl_usd':         [7, 30],
    'apy_7d':          [7, 30],
}

def _stable_hash_0_1(s: str, mod: int = 1000) -> float:
    """Deterministic hash to [0,1) based on md5."""
    h = hashlib.md5(s.encode('utf-8')).hexdigest()
    val = int(h[:8], 16) % mod
    return val / float(mod)

def build_pool_feature_row(panel_df: pd.DataFrame,
                           pool_id: str,
                           asof: pd.Timestamp,
                           group_col: str = GROUP_COL) -> dict:
    """
    Build leakage-safe feature row for (pool_id, asof) using only information
    available at end-of-day `asof`, for forecasting next-day actual_apy.

    - Uses preprocess_data(..., exogenous_cols=EXOG_BASE) which creates *_shifted
      versions of exogenous vars to avoid leakage.
    - Adds explicit lags from LAG_SETS.
    - Pulls neighbor features computed on `panel_df` for the same day (t) with *_nbr names.
    """

    # normalize tz
    asof = pd.Timestamp(asof)
    asof = asof.tz_localize('UTC') if asof.tz is None else asof.tz_convert('UTC')
    asof_day = asof.normalize()

    # history for this pool up to asof
    hist = (panel_df.loc[panel_df['pool_id'] == pool_id]
                    .sort_values('date')
                    .copy())

    if hist.empty:
        return {}

    # ensure datetime normalized
    hist['date'] = pd.to_datetime(hist['date'], utc=True).dt.normalize()
    hist = hist.set_index('date').sort_index()

    # require the asof day to be present (features at t to predict t+1)
    if asof_day not in hist.index:
        return {}

    # ---- base + exogenous (with _shifted from preprocess_data) ----
    feat = preprocess_data(hist.reset_index(), exogenous_cols=EXOG_BASE)

    # add explicit lags
    for col, lags in LAG_SETS.items():
        if col in feat.columns:
            feat = create_lagged_features(feat, col, lags)

    # keep only the row at `asof`
    if asof_day not in feat.index:
        return {}
    row = feat.loc[asof_day].to_dict()

    # ---- neighbor features on the same date (already computed on panel_df) ----
    # names align with add_neighbor_features() version I shared
    nbr_cols = [
        'group_tvl_sum_t_nbr',
        'group_apy_mean_t_nbr',
        'group_apy_median_t_nbr',
        'group_apy_std_t_nbr',
        'tvl_share_nbr',
        'apy_rank_nbr',
        'grp_ex_mean_t_nbr',
        'grp_ex_mean_7d_nbr',
        'group_tvl_sum_t_nbr_lag1',
        'group_apy_mean_t_nbr_lag1',
        'group_apy_median_t_nbr_lag1',
        'group_apy_std_t_nbr_lag1',
        'grp_ex_mean_t_nbr_lag1',
        'grp_ex_mean_7d_nbr_lag1',
    ]

    day_pool = panel_df[
        (panel_df['pool_id'] == pool_id) &
        (pd.to_datetime(panel_df['date'], utc=True).dt.normalize() == asof_day)
    ]

    # copy numeric nbr features if present; keep group_col raw (string) if requested
    for c in nbr_cols:
        if c in day_pool.columns and len(day_pool) > 0 and pd.notna(day_pool[c].iloc[0]):
            try:
                row[c] = float(day_pool[c].iloc[0])
            except Exception:
                # if it happens to be non-numeric (shouldn't), set 0.0
                row[c] = 0.0
        else:
            row[c] = 0.0

    # keep the group label (categorical) as-is if you want to feed it to LGBM
    if group_col:
        if group_col in day_pool.columns and len(day_pool) > 0:
            row[group_col] = day_pool[group_col].iloc[0]
        else:
            row[group_col] = 'ALL'

    # ---- calendar (no leakage) ----
    dow = int(asof_day.dayofweek)
    doy = int(asof_day.dayofyear)
    row['dow_sin'] = np.sin(2*np.pi * dow / 7.0)
    row['dow_cos'] = np.cos(2*np.pi * dow / 7.0)
    row['doy_sin'] = np.sin(2*np.pi * doy / 365.25)
    row['doy_cos'] = np.cos(2*np.pi * doy / 365.25)

    # ---- stable pool id hash (0..1) ----
    row['pool_id_hash'] = _stable_hash_0_1(pool_id, mod=1000)

    return row

In [27]:

def build_global_panel_dataset(asof_start: pd.Timestamp, asof_end: pd.Timestamp,
                               pool_ids: list, group_col: str = GROUP_COL,
                               hist_days:int=HIST_DAYS_PANEL) -> pd.DataFrame:
    """
    Build training rows with target = next-day actual_apy.
    Only include pools that have >=3 valid actual_apy values by `asof` (cold-start guard).
    """
    rows = []
    days = pd.date_range(asof_start, asof_end, freq='D')
    engine = get_db_connection()

    for t in days:
        t = pd.Timestamp(t)
        t = t.tz_localize('UTC') if t.tz is None else t.tz_convert('UTC')

        panel = fetch_panel_history(t, pool_ids, days=hist_days, group_col=group_col)
        if panel.empty:
            continue
        panel = add_neighbor_features(panel, group_col=group_col)

        # realized next day (target)
        with engine.connect() as conn:
            realized = pd.read_sql(
                text("""SELECT pool_id, date, actual_apy
                        FROM pool_daily_metrics
                        WHERE date = :d"""),
                conn, params={"d": (t + pd.Timedelta(days=1)).normalize().to_pydatetime()}
            )
        if realized.empty:
            continue
        realized['date'] = pd.to_datetime(realized['date'], utc=True).dt.normalize()

        pools_today = panel.loc[panel['date'] == t.normalize(), 'pool_id'].unique().tolist()
        for pid in pools_today:

            n_hist = _count_actual_history(panel, pid, t.normalize())
            # inside the for pid loop
            hist_apys = panel.loc[
                (panel['pool_id'] == pid) &
                (panel['apy_7d'].notna()) &
                (panel['date'] <= t)
            ]['apy_7d']

            n_valid = len(hist_apys)
            if n_hist < 2:
                continue  # do not train on unstable early windows
            elif n_valid == 2:
                # simple average baseline
                baseline = hist_apys.mean()
                rows.append({
                    'pool_id': pid,
                    'asof': t.normalize(),
                    'target_apy_t1': np.nan,   # not used for training
                    'pred_global_apy': baseline,
                    'cold_start_flag': True
                })
                continue

            feat_row = build_pool_feature_row(panel, pid, t, group_col=group_col)
            if not feat_row:
                continue

            y_next = realized.loc[realized['pool_id'] == pid, 'actual_apy']
            target = float(y_next.iloc[0]) if len(y_next) > 0 and pd.notna(y_next.iloc[0]) else np.nan
            if pd.isna(target):
                continue

            feat_row.update({
                'pool_id': pid,
                'asof': t.normalize(),
                'target_apy_t1': target
            })
            rows.append(feat_row)

    return pd.DataFrame(rows)

In [28]:
def fit_global_panel_model(panel_df: pd.DataFrame):
    """
    Fits a single global model for APY_{t+1} (next-day actual_apy).
    Returns (model, feat_cols)
    """
    df = panel_df.copy()

    # Clean basic issues
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['target_apy_t1'])

    # Select features (drop identifiers / time / target)
    drop = {'target_apy_t1', 'pool_id', 'asof'}
    feat_cols = [c for c in df.columns if c not in drop]

    # Keep only numeric feature columns to avoid cat-feature mismatches
    numeric_cols = [c for c in feat_cols if pd.api.types.is_numeric_dtype(df[c])]
    if len(numeric_cols) < len(feat_cols):
        # optional: log which were dropped
        # dropped = sorted(set(feat_cols) - set(numeric_cols))
        feat_cols = numeric_cols

    if not feat_cols:
        raise ValueError("No numeric features available to train the global model.")

    X = df[feat_cols].astype(float)
    y = df['target_apy_t1'].astype(float)

    if len(df) < MIN_ROWS_PANEL:
        print(f"⚠️ Not enough panel rows ({len(df)}). Model may be weak.")

    if _USE_LGBM:
        model = lgb.LGBMRegressor(
            objective='regression',
            n_estimators=800,
            num_leaves=64,
            learning_rate=0.03,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=123
        )
        model.fit(X, y)
    else:
        model = XGBRegressor(
            n_estimators=900, max_depth=6, learning_rate=0.04,
            subsample=0.9, colsample_bytree=0.9, n_jobs=-1, random_state=123
        )
        model.fit(X, y)

    return model, feat_cols


def predict_global_lgbm_head(asof: pd.Timestamp, pool_ids: list,
                             model, feat_cols: list,
                             group_col: str = GROUP_COL,
                             hist_days:int=HIST_DAYS_PANEL) -> pd.DataFrame:
    """
    Predict APY_{t+1} at `asof` with cold-start rules:
      - <2 actual points: skip prediction
      - =2 actual points: emit baseline (mean of last 2 actual_apy), flag cold_start
      - >=3: model prediction
    """
    t = pd.Timestamp(asof)
    t = t.tz_localize('UTC') if t.tz is None else t.tz_convert('UTC')

    panel = fetch_panel_history(t, pool_ids, days=hist_days, group_col=group_col)
    if panel.empty:
        return pd.DataFrame(columns=['pool_id','target_date','pred_global_apy','cold_start_flag'])

    panel = add_neighbor_features(panel, group_col=group_col)
    pools_today = panel.loc[panel['date'] == t.normalize(), 'pool_id'].unique().tolist()

    out_rows = []
    for pid in pools_today:
        n_hist = _count_actual_history(panel, pid, t.normalize())

        # 0–1: skip
        if n_hist < 3:
            continue

        # 2: baseline
        if n_hist == 2:
            base = _baseline_from_actual(panel, pid, t.normalize())
            if base is not None:
                out_rows.append({
                    'pool_id': pid,
                    'target_date': (t + pd.Timedelta(days=1)).date(),
                    'pred_global_apy': float(base),
                    'cold_start_flag': True
                })
            continue

        # >=3: model
        f = build_pool_feature_row(panel, pid, t, group_col=group_col)
        if not f:
            continue
        x = {c: f.get(c, 0.0) for c in feat_cols}
        x_df = pd.DataFrame([x], columns=feat_cols).astype(float)
        pred = float(model.predict(x_df)[0])
        out_rows.append({
            'pool_id': pid,
            'target_date': (t + pd.Timedelta(days=1)).date(),
            'pred_global_apy': pred,
            'cold_start_flag': False
        })

    return pd.DataFrame(out_rows)

In [29]:
def get_filtered_pool_ids_readonly(limit=None):
    # Your function in fp.get_filtered_pool_ids() uses the same SQL; this read-only helper
    # avoids accidentally importing the writer in main().
    engine = get_db_connection()
    sql = """
    SELECT DISTINCT pool_id
    FROM pool_daily_metrics
    WHERE is_filtered_out = FALSE
    """
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn)
    ids = df["pool_id"].tolist()
    return ids[:limit] if limit else ids

In [30]:
def fetch_realized_for_date(target_date, group_col=None):
    """
    Fetch realized (actual) APY and TVL for all pools on a specific date.

    Parameters
    ----------
    target_date : datetime.date or pd.Timestamp
        The date for which realized values (t+1 actuals) are fetched.
    group_col : str, optional
        Optional grouping column to include (e.g., 'pool_group').

    Returns
    -------
    pd.DataFrame
        Columns: ['pool_id', 'date', 'actual_apy', 'actual_tvl', group_col (optional)]
    """
    target_date = pd.Timestamp(target_date)
    target_date = target_date.tz_localize('UTC') if target_date.tz is None else target_date.tz_convert('UTC')
    target_day = target_date.normalize().to_pydatetime()

    engine = get_db_connection()
    gsel = f", {group_col}" if group_col else ""
    sql = f"""
        SELECT 
            pool_id,
            date,
            actual_apy,
            actual_tvl
            {gsel}
        FROM pool_daily_metrics
        WHERE date = %(d)s
          AND is_filtered_out = FALSE
    """

    with engine.connect() as conn:
        df = pd.read_sql(sql, conn, params={'d': target_day})

    if df.empty:
        return df

    # Normalize timezone and date type
    df['date'] = pd.to_datetime(df['date'], utc=True).dt.normalize()
    df.rename(columns={
        'actual_apy': 'realized_apy',
        'actual_tvl': 'realized_tvl'
    }, inplace=True)
    return df

In [31]:
TRAIN_WINDOW = 60

END_DATE   = pd.Timestamp.utcnow().normalize() - pd.Timedelta(days=5) 
START_DATE = END_DATE - pd.Timedelta(days=TRAIN_WINDOW) 


In [37]:
END_DATE

Timestamp('2025-11-02 00:00:00+0000', tz='UTC')

In [ ]:

pool_ids = get_filtered_pool_ids_readonly(limit=200)  # your helper

# 1) Build panel training on a rolling window
panel_train = build_global_panel_dataset(
    asof_start=START_DATE,
    asof_end=END_DATE,
    pool_ids=pool_ids,
    group_col=GROUP_COL,
    hist_days=HIST_DAYS_PANEL
)
print("Panel rows:", len(panel_train))

# 2) Fit the global model
glob_model, glob_feats = fit_global_panel_model(panel_train)
print("Global model trained. #features:", len(glob_feats))


pred_global = predict_global_lgbm_head(
    asof = END_DATE, # the day to trigger the model to predict on END_DATE + 1 day
    pool_ids=pool_ids,
    model=glob_model,
    feat_cols=glob_feats,
    group_col=GROUP_COL,
    hist_days=HIST_DAYS_PANEL
)


Panel rows: 1884
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4104
[LightGBM] [Info] Number of data points in the train set: 1862, number of used features: 38
[LightGBM] [Info] Start training from score 11.392993
Global model trained. #features: 48


In [38]:
pred_global

,pool_id,target_date,pred_global_apy,cold_start_flag
0,088d7b29-d111-4572-b700-56e4fe39515e,2025-11-03,8.156957,False
1,09348d64-2d1d-4ab1-ba54-dcd78a783e79,2025-11-03,11.242616,False
2,0aeae5ff-b988-4127-b682-2910e3950d41,2025-11-03,7.969124,False
3,0b3c155c-7db7-48f5-80a3-edf1186becc2,2025-11-03,6.638796,False
4,0d4f7043-c27c-4b32-8283-234ee409a317,2025-11-03,8.985501,False
5,256cca8a-b88a-47d6-9297-a47858cccbb3,2025-11-03,11.285655,False
6,2cac4020-2809-4136-8609-91b91ffeef2c,2025-11-03,13.292908,False
7,362ff642-4b12-4292-b0ca-5cd22f6f3ed0,2025-11-03,13.647794,False
8,38906d44-1807-469b-ba7f-ba1d65ebabd5,2025-11-03,10.841037,False
9,444745ee-4577-4a84-8ee3-dfda21367af9,2025-11-03,5.356840,False
